# Model evaluation

## Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from google_drive_downloader import GoogleDriveDownloader as gdd
import shutil, os
from ops import load_opt, load_sar, generate_save_patches, min_max_scaler, recall_precision, compute_mAP
import numpy as np
import logging
from PIL import Image
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pickle
import numpy as np
import skimage.morphology
from sklearn.metrics import confusion_matrix
from model.models import Model_1, Model_2
from model.losses import FocalLoss, WBCE
from tensorflow.keras.layers import Input
from dataloader import DataLoader
from tensorflow.keras.optimizers.schedules import InverseTimeDecay



## Load Model

In [3]:
# load the params-patches.json options
with open(os.path.join('v1', 'params-patches.json')) as param_file:
    params_patches = json.load(param_file)

# load the params-patches.json options
with open(os.path.join('v1', 'params-training.json')) as param_file:
    params_training = json.load(param_file)
    
#load the params-model.json options
with open(os.path.join('v1', 'params-model.json')) as param_file:
    params_model = json.load(param_file)

#load the shapes.json options
with open('shapes.json') as param_file:
    shapes_json = json.load(param_file)

In [4]:
#patches_path = params_patches['patches_path']

#train_path = os.path.join(patches_path, params_patches['train_sub'])
#val_path = os.path.join(patches_path, params_patches['val_sub'])
#test_path = os.path.join(patches_path, params_patches['test_sub'])
#full_path = params_patches['full_path']

img_path = params_patches['img_path']
data_raw = os.path.join(img_path, params_patches['data_sub']) 
label_raw = os.path.join(img_path, params_patches['label_sub'])

pred_path = params_patches['pred_path']

In [5]:
model = Model_2(name='modelo_1')

metrics = {
}

weights = [0.2, 0.8, 0.0]


optimizers = {
    'opt': tf.keras.optimizers.Adam(learning_rate = 1e-4),
    'sar': tf.keras.optimizers.Adam(learning_rate = 1e-4),
    'fusion': tf.keras.optimizers.Adam(learning_rate = 1e-4),
}

class_indexes = [0, 1]

model.compile(
    optimizers = optimizers,
    loss_fn = WBCE,
    metrics_dict = metrics,
    class_weights = weights,
    class_indexes = class_indexes,
    run_eagerly=params_training['run_eagerly']
)

model.build(
    input_shape = [
        (None, params_patches['patch_size'], params_patches['patch_size'], params_model['opt_channels']),
        (None, params_patches['patch_size'], params_patches['patch_size'], params_model['sar_channels'])
    ]
    )
model.load_weights('weights.h5')

## Eval Model

### Load images

In [6]:
opt = np.load(os.path.join(data_raw, 'opt.npy'))
sar = np.load(os.path.join(data_raw, 'sar.npy'))

### Make Predictions

In [ ]:
pred = model.predict_from_patches(
    (opt, sar), 
    params_patches['patch_size'], 
    params_patches['patch_stride'], 
    16)

  0%|          | 0/9990 [00:00<?, ?it/s]

In [ ]:
labels = np.load(os.path.join(label_raw, 'labels.npy'))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=5, figsize = (25,8))


img = Image.fromarray(np.uint8(to_categorical(labels, 3)[:,:,1]*255))
ax[0].axis('off')
ax[0].set_title(f'Label')
ax[0].imshow(img, cmap = 'gray')

img = Image.fromarray(np.squeeze(np.uint8(pred[0]*255)))
ax[1].axis('off')
ax[1].set_title(f'OPT')
ax[1].imshow(img, cmap = 'gray')

img = Image.fromarray(np.squeeze(np.uint8(pred[1]*255)))
ax[2].axis('off')
ax[2].set_title(f'SAR')
ax[2].imshow(img, cmap = 'gray')

img = Image.fromarray(np.squeeze(np.uint8(pred[2]*255)))
ax[3].axis('off')
ax[3].set_title(f'FUSION')
ax[3].imshow(img, cmap = 'gray')

img = Image.fromarray(np.squeeze(np.uint8(pred[3]*255)))
ax[4].axis('off')
ax[4].set_title(f'COMBINED')
ax[4].imshow(img, cmap = 'gray')

fig.show()

In [ ]:
shutil.rmtree(pred_path, ignore_errors=True)
os.makedirs(pred_path)

In [ ]:
np.save(os.path.join(pred_path, 'opt.npy'), np.squeeze(pred[0]))
np.save(os.path.join(pred_path, 'sar.npy'), np.squeeze(pred[1]))
np.save(os.path.join(pred_path, 'fusion.npy'), np.squeeze(pred[2]))
np.save(os.path.join(pred_path, 'combination.npy'), np.squeeze(pred[3]))

In [ ]:
test_mask = np.squeeze(np.zeros_like(pred[0]))
test_tiles = params_patches['test_tiles']
split_v = params_patches['tiles_v']
split_h = params_patches['tiles_h']
lines = np.array_split(range(pred[0].shape[0]), split_v)
cols = np.array_split(range(pred[0].shape[1]), split_h)
i = 0
dic = {}
for line in lines:
    for col in cols:
        i+=1
        if i not in test_tiles:
            continue
        test_mask[line[0]:line[-1]+1, col[0]:col[-1]+1] = 1
img = Image.fromarray(np.uint8(test_mask*255))
resize = 0.02
shape = (int(resize*test_mask.shape[1]), int(resize*test_mask.shape[0]))
img = img.resize(shape)
img

In [ ]:
rec_prec_l = []
map_l = []
for i in range(1):
    ref1 = np.ones_like(labels).astype(np.float32)

    ref1 [labels == 2] = 0
    TileMask = test_mask * ref1
    GTTruePositives = labels==1

    Npoints = 10
    Pmax = np.max(pred[i][GTTruePositives * TileMask ==1])
    ProbList = np.linspace(Pmax,0,Npoints) 
    
    rec_pre_metrics = recall_precision(ProbList, labels, np.squeeze(pred[i]), 625, test_mask)
    rec_prec_l.append(rec_pre_metrics)
    
    Recall = rec_pre_metrics[:,0]
    Precision = rec_pre_metrics[:,1]

    mAP = compute_mAP(Recall, Precision)
    map_l.append(mAP)
    print(f'mAP({i}): ', mAP)
    
    

In [ ]:
gragh_labels = {
    0: "Optical",
    1: "SAR",
    2: "Fusion",
    3: "Combined"
}
gragh_style = {
    0: "r-",
    1: "g-",
    2: "b-",
    3: "k-"
}

plt.figure(figsize = (10,10))
for i in range(1):
    plt.plot(rec_prec_l[i][:,0], rec_prec_l[i][:,1], gragh_style[i], label=gragh_labels[i])


plt.legend(loc='upper right')
plt.grid()
plt.show()